# Google cloud 

Brief review from 203B: 

- Using a collection of remote servers for computation, data storage/manipulation, etc.

- Pay for clock-cycles, storage, and network flow rather than hardware.

- Computing when you need it:

  - Scalable computing
  - Scalable storage
  
Why is it popular?

- Scalability!

- Adapt to fluctuating demand:

  - Websites with fluctuating traffic.

  - Large corporations use much more computing during business hours than off-business hours.

- Efficiency:

  - Pay for what you need.

  - No need for hardware maintenance.

  - Less waiting for fixed compute-time jobs.
  
  
Last quarter, the Compute Engine and BigQuery were used. 



You chose a virtual machine with fixed number of cores and memory, launched it, and ran LINUX commands on the machine. Carefully choosing the optimal type of instances might not be that simple, there literally are hundreds of available options! 

Well, believe it or not, with Spark, you might not even need to do that. [Dataproc Serverless](https://cloud.google.com/dataproc-serverless/docs) allows you to run PySpark jobs without needing to configure infrastructure and autoscaling. The service will do that for you depending on the amount of the data. Dataproc Serverless supports PySpark batch workloads and sessions / notebooks.

Similar thing is also provided in Amazon Web Servies, under the name of "Amazon EMR".


## Getting Started with Dataproc Serverless

https://cloud.google.com/dataproc-serverless/docs/quickstarts/jupyterlab-sessions

https://codelabs.developers.google.com/dataproc-serverless

### 1. Project creation

Visit cloud.google.com, and on the top will be a drop-down list of projects. Click on it, then click "New Project". Creat a new project, e.g., with the name Biostat-203c. 

### 2. Select correct Billing Accounts

Either select "Billing" from the navigation menu (a button on the top-left) -OR- 
On the search bar on the top, type "Billing Accounts", and click on that menu. Make sure to select "Billing Account for Education". 

### 3. Enable the `BigQuery` and `Dataproc` APIs.

On the search bar, type "Cloud Dataproc API", then click on the top search result. Click the "Enable" button. Repeat for "BigQuery API".

-OR-

on your terminal:

```bash
gcloud services enable bigquery.googleapis.com
gcloud services enable dataproc.googleapis.com
```

### 4. Install Gcloud CLI.

https://cloud.google.com/sdk/docs/install



### 5. Initialize Gcloud CLI.

After installing the Gcloud CLI, run `gcloud init` on your terminal. You will be promped to log in through Google and to select a project. 

### 6. Install the Dataproc JupyterLab plugin

__On your BIOSTAT-203C conda environment,__ run:
```bash
pip install dataproc-jupyter-plugin
```
Enable the plugin extension:

```bash
jupyter server extension enable dataproc_jupyter_plugin
```
Then, restart your Jupyter Lab:
```bash
jupyter lab
```

Then, the plugin will be loaded. 


The JupyterLab Launcher page opens in your browser. It contains a Dataproc Jobs and Sessions section. It can also contain Dataproc Serverless Notebooks and Dataproc Cluster Notebooks sections if you have access to Dataproc serverless notebooks or Dataproc clusters with the Jupyter optional component running in your project.


### 7. Select the region
From the menu of JupyterLab, go to  Settings > Google Dataproc settings. 

Select a region. e.g., `us-west1` for Oregon, further away from here compared to `us-west2` (Los Angeles), but often is cheaper.

You will need to restart Jupyter Lab again. 


### 8. Create a Serverless Runtime Template

Fill in template name and description. 

__Network Configuration__: The subnetwork must have Private Google Access enabled and must allow subnet communication on all ports (see Dataproc Serverless for Spark network configuration).

If the `default` network's subnet for the region you configured when you ran `gcloud init` in Before you begin is not enabled for Private Google Access:
- [Enable it for Private Google Access](https://cloud.google.com/vpc/docs/configure-private-google-access#enabling-pga), or
- Select another network with a regional subnetwork that has Private Google Access enabled. You can change the region that Dataproc Serverless uses from the JupyterLab Settings > Dataproc Settings page.


#### 8-a. 
To enable the `default` network for private google access, go to VPC networks page, then click on `default` network, then click on the `SUBNETS` tab. Then click on `default` next to the desired region (e.g. `us-west1`). Then you should be on "Subnet Details" page. Click on "Edit" button, then there is "Private Google Access" On/Off. Turn it on and press the "SAVE" button. -OR-

```bash 
gcloud compute networks subnets update default \
  --region=us-west1 \
  --enable-private-ip-google-access
```


For other parameters, see the link above. 


### 9. Create a storage bucket
Create a storage bucket that will be used to store assets created in this codelab.

Choose a name for your bucket. [Bucket names must be globally unique across all users](https://cloud.google.com/storage/docs/buckets#naming).


```bash
export BUCKET=<your-bucket-name>
```
Create the bucket in the region you intend to run your Spark jobs.

```bash
gsutil mb -l us-west1 gs://${BUCKET}
```
You can see that your bucket is available in the Cloud Storage console. You can also run `gsutil ls` to see your bucket.

### 10. Now, let's run Dataproc pyspark runtime from the Launcher. 

Let's try these two code snippets:



A code snippet for estimating the value of $\pi$ via MCMC. 

```python
%%time
import random
    
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1
    
count = sc.parallelize(range(0, 100000)) .filter(inside).count()
print("Pi is roughly %f" % (4.0 * count / 100000))
```


The New York City (NYC) Citi Bike Trips public dataset from BigQuery. NYC Citi Bikes is a paid bike sharing system within NYC. You will perform some simple transformations and print the top ten most popular Citi Bike station ids.

```python
import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import BooleanType

bucket = "XXXXXXX" # Your google storage bucket name. 
table = "bigquery-public-data:new_york_citibike.citibike_trips"

spark = SparkSession.builder \
          .appName("pyspark-example") \
          .config("spark.jars","gs://spark-lib/bigquery/spark-bigquery-with-dependencies_2.12-0.26.0.jar") \
          .getOrCreate()

df = spark.read.format("bigquery").load(table)

top_ten = df.filter(col("start_station_id") \
            .isNotNull()) \
            .groupBy("start_station_id") \
            .count() \
            .orderBy("count", ascending=False) \
            .limit(10) \
            .cache()

top_ten.show()

top_ten.write.option("header", True).csv(f"gs://{bucket}/citibikes_top_ten_start_station_ids")
```
